In [11]:
import numpy as np 
import pandas as pd 
import seaborn as sb
import xgboost as xgb
import math

from subprocess import check_output


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory



train = pd.read_csv("/home/amal/Bureau/train.csv")
test = pd.read_csv("/home/amal/Bureau/test.csv")

#next we create training and testing sets
X_train = train.drop(["id", "target"], axis=1)
Y_train = train["target"].copy()
X_test = test.drop("id", axis = 1).copy()

from sklearn.model_selection import StratifiedKFold

params = {"objective": "multi:softprob", "eval_metric":"mlogloss", "num_class": 9}

#perform K-fold validation with k = 10
#here you should add the loop 
iterations = [110 , 120 , 130 , 140 , 150 , 160 , 170 , 180]
for iteration in iterations:
    skf = StratifiedKFold(n_splits=10)
    score = 0
    for train, test in skf.split(X_train, Y_train):
        X_train_k = X_train.iloc[train]
        Y_train_k = Y_train.iloc[train]
        X_test_k = X_train.iloc[test]
        Y_test_k = Y_train.iloc[test] 
        T_train_xgb = xgb.DMatrix(X_train_k, Y_train_k)
        X_test_xgb  = xgb.DMatrix(X_test_k)
        #number of iterations - initial 20, 
        gbm = xgb.train(params, T_train_xgb, iteration)
        Y_pred = gbm.predict(X_test_xgb)
        val_score =0
        Y_k = Y_test_k.as_matrix()
        for i in range(len(Y_pred)):
            if( Y_k[i] == Y_pred[i].argmax()):
                val_score +=1
        val_score = val_score/len(Y_pred)
        score += val_score
    score1 = score/10
    print (iteration)
    print (score1)

#Make a submission based on model "sample_submission.csv"

submission = pd.DataFrame({ "id": test["id"]})

i = 0

for num in class_range:
    col_name = str("Class_{}".format(num))
    submission[col_name] = Y_pred[:,i]
    i = i + 1
    
submission.to_csv('/home/amal/Bureau/otto_final.csv', index=False)

XGBoostError: [00:16:41] src/objective/multiclass_obj.cc:42: Check failed: (info.labels.size()) != (0) label set cannot be empty